## Exercise: Optional Monoid

Write the `Monoid` instance for our `Maybe` type renamed to `Optional`.

In [12]:
import Data.Monoid

data Optional a =
    Nada
    | Only a
    deriving (Eq, Show)

instance Monoid a => Monoid (Optional a) where
    mempty  = Nada
    
    mappend Nada Nada         = Nada
    mappend Nada (Only x)     = Only x
    mappend (Only x) Nada     = Only x
    mappend (Only x) (Only y) = Only (x <> y)
    
--

Only (Sum 1) `mappend` Only (Sum 1)
Only (Product 4) `mappend` Only (Product 2)
Only (Sum 1) `mappend` Nada
Only [1] `mappend` Nada
Nada `mappend` Only (Sum 1)

Only (Sum {getSum = 2})

Only (Product {getProduct = 8})

Only (Sum {getSum = 1})

Only [1]

Only (Sum {getSum = 1})

## Exercise: Madness

In [11]:
import Data.Monoid

type Verb = String
type Adjective = String
type Adverb = String
type Noun = String
type Exclamation = String

madlibbin' :: Exclamation -> Adverb -> Noun -> Adjective -> String
madlibbin' e adv noun adj =
    e <> "! he said " <>
    adv <> " as he jumped into his car " <>
    noun <> " and drove off with his " <>
    adj <> " wife."
    
madlibbinBetter' :: Exclamation -> Adverb -> Noun -> Adjective -> String
madlibbinBetter' e adv noun adj =
    mconcat [
        e
        , "! he said "
        , adv
        , " as he jumped into his car "
        , noun
        , " and drove off with his "
        , adj
        , " wife."
     ]

--

madlibbin' "Yo" "loudly" "Renee" "unique"
madlibbinBetter' "Yo" "loudly" "Renee" "unique"

"Yo! he said loudly as he jumped into his car Renee and drove off with his unique wife."

"Yo! he said loudly as he jumped into his car Renee and drove off with his unique wife."

## Exercise: Maybe Another Monoid

Write a `Monoid` instance for a `Maybe` type which doesn’t require a `Monoid` for the contents. Reuse the `Monoid` law QuickCheck properties and use them to validate the instance.

Don’t forget to write an `Arbitrary` instance for `First'`. We won’t always stub that out explicitly for you. We suggest learning how to use the `frequency` function from QuickCheck for `First'`’s instance.

In [27]:
import Data.Monoid
import Test.QuickCheck

newtype First' a =
    First' { getFirst' :: Optional a }
    deriving (Eq, Show)

instance Arbitrary a => Arbitrary (First' a) where
    arbitrary = do
        x <- arbitrary
        frequency [(3, return (First' (Only x)))
                  ,(1, return (First' Nada)) ]

instance Monoid (First' a) where
    mempty = First' Nada
    
    mappend (First' Nada) f = f
    mappend f (First' Nada) = f
    mappend f _ = f

firstMappend :: First' a -> First' a -> First' a
firstMappend = mappend

type FirstMappend = First' String -> First' String -> First' String -> Bool
type FstId = First' String -> Bool

--

monoidAssoc :: (Eq m, Monoid m) => m -> m -> m -> Bool
monoidAssoc a b c = (a <> (b <> c)) == ((a <> b) <> c)

monoidLeftIdentity :: (Eq m, Monoid m) => m -> Bool
monoidLeftIdentity a = (mempty <> a) == a

monoidRightIdentity :: (Eq m, Monoid m) => m -> Bool
monoidRightIdentity a = (a <> mempty) == a

--

quickCheck (monoidAssoc :: FirstMappend)
quickCheck (monoidLeftIdentity :: FstId)
quickCheck (monoidRightIdentity :: FstId)
    
First' (Only 1) `mappend` First' Nada
First' Nada `mappend` First' Nada
First' Nada `mappend` First' (Only 2)
First' (Only 1) `mappend` First' (Only 2)

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

First' {getFirst' = Only 1}

First' {getFirst' = Nada}

First' {getFirst' = Only 2}

First' {getFirst' = Only 1}